In [53]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings

In [60]:
df = pd.read_csv('data/train.csv')
# train1 is numerical data types
trainY = df['SalePrice']
df.drop(['SalePrice'], axis=1, inplace=True)
train1 = df.select_dtypes(exclude=['object'])
# train2 is categorical data types
train2 = df.select_dtypes(['object'])


In [78]:
trainY[2]

223500